This notebook is a sample code with Japanese comments.

# 3.3 Titanicの先へ行く③！　テキストデータに触れてみよう

テーブルデータと共通する部分
- 機械学習の教師あり学習
    - 学習用データセットの特徴量、目的変数の対応関係を機械学習アルゴリズムで学習して未知のデータセットに対する性能を得る

そのままのテキストデータでは機械学習アルゴリズムで扱えない

何かしらでベクトルに変換する必要がある

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame({'text': ['I like kaggle very much',
                            'I do not like kaggle',
                            'I do really love machine learning']})
df

,text
0,I like kaggle very much
1,I do not like kaggle
2,I do really love machine learning


# Bag of Words
文章で登場した単語の回数を数える方法

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# 配列の1要素が1文に対応
# 1文目を抜粋
# 元の文章: I like kaggle very much
# 変換後の配列: [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1]
# インデックス: {'i': 1, 'like': 4, 'kaggle': 2, 'very': 10, 'much': 7, 'do': 0, 'not': 8, 'really': 9, 'love': 5, 'machine': 6, 'learning': 3}
# 'i', 'kaggle', 'like', 'much', 'very'が文内にそれぞれ1つ存在することが分かる
vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
bag = vectorizer.fit_transform(df['text'])
bag.toarray()

array([[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]])

In [7]:
print(vectorizer.vocabulary_)

{'i': 1, 'like': 4, 'kaggle': 2, 'very': 10, 'much': 7, 'do': 0, 'not': 8, 'really': 9, 'love': 5, 'machine': 6, 'learning': 3}


# TF-IDF
単語の珍しさを考慮したベクトル化の手法

Term Frequency(単語の登場回数)だけではなく、Inverse Document Frequency(ドキュメント内での登場回数の頻度の逆数)を掛け合わせる

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Bag of Wordsの配列と比較
# Bag of Words: [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1]
# TF-IDF: [0, 0.31544415, 0.40619178, 0, 0.40619178, 0, 0, 0.53409337, 0, 0, 0.53409337]
# 0の位置と0より大きい値の位置は同じ
# 0か1の離散値ではなく、0~1の連続値を取るようになった
# 単語の珍しさに応じて大きな値になっている
vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer = TfidfTransformer()

tf = vectorizer.fit_transform(df['text'])
tfidf = transformer.fit_transform(tf)
print(tfidf.toarray())

[[0.         0.31544415 0.40619178 0.         0.40619178 0.
  0.         0.53409337 0.         0.         0.53409337]
 [0.43306685 0.33631504 0.43306685 0.         0.43306685 0.
  0.         0.         0.56943086 0.         0.        ]
 [0.34261996 0.26607496 0.         0.45050407 0.         0.45050407
  0.45050407 0.         0.         0.45050407 0.        ]]


In [11]:
print(vectorizer.vocabulary_)

{'i': 1, 'like': 4, 'kaggle': 2, 'very': 10, 'much': 7, 'do': 0, 'not': 8, 'really': 9, 'love': 5, 'machine': 6, 'learning': 3}


# Word2vec

単語同士の意味的な近さを考慮したベクトル化手法

In [13]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 42.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 20.5 MB/s eta 0:00:00


In [16]:
from gensim.models import word2vec

# 学習
sentences = [d.split() for d in df['text']]
model = word2vec.Word2Vec(sentences, vector_size=10, min_count=1, window=2, seed=7)

In [17]:
# 学習した単語はベクトル形式に変換できる
model.wv['like']

array([ 0.01650858,  0.01069946,  0.00188946,  0.09910005,  0.06153275,
        0.05853238,  0.04005488,  0.02443584, -0.03179482,  0.09779203],
      dtype=float32)

In [18]:
# 学習に用いた単語の中から似ている単語を抽出できる
# 注意: 今回はデータセットが3文しかないので十分に意味の近さを学習できていない
model.wv.most_similar('like')

[('I', 0.4254004955291748),
 ('machine', 0.36355969309806824),
 ('not', 0.311229407787323),
 ('kaggle', -0.004140505567193031),
 ('much', -0.11530754715204239),
 ('do', -0.1529017835855484),
 ('love', -0.25542783737182617),
 ('really', -0.4161785840988159),
 ('learning', -0.44330498576164246),
 ('very', -0.4433840215206146)]

In [20]:
df['text'][0].split()

['I', 'like', 'kaggle', 'very', 'much']

In [21]:
import numpy as np

wordvec = np.array([model.wv[word] for word in df['text'][0].split()])
wordvec

array([[ 0.08898099,  0.02501909,  0.03683598,  0.07944275,  0.01565849,
         0.05513714,  0.0667302 , -0.05495857, -0.08889369, -0.03996675],
       [ 0.01650858,  0.01069946,  0.00188946,  0.09910005,  0.06153275,
         0.05853238,  0.04005488,  0.02443584, -0.03179482,  0.09779203],
       [ 0.06329302, -0.03939352, -0.03167932, -0.04431488,  0.04389417,
        -0.04902608,  0.09809195, -0.01098474, -0.00437022,  0.00090965],
       [ 0.03720424, -0.02774719,  0.02864924,  0.01963681, -0.07835456,
        -0.08814968,  0.03203132, -0.02247364,  0.01966591, -0.03539274],
       [-0.09157717,  0.04835419, -0.00529734, -0.08170088, -0.05110302,
         0.00822875,  0.04535742,  0.00155444,  0.02258943,  0.07426786]],
      dtype=float32)

In [23]:
# 文に登場する単語ベクトルの平均を取る
np.mean(wordvec, axis=0)

array([ 0.02288193,  0.00338641,  0.0060796 ,  0.01443277, -0.00167443,
       -0.0030555 ,  0.05645315, -0.01248533, -0.01656068,  0.01952201],
      dtype=float32)

In [24]:
# 文に登場する単語ベクトルの各要素の最大値を取る
# SWEM-maxと呼ばれる
np.max(wordvec, axis=0)

array([0.08898099, 0.04835419, 0.03683598, 0.09910005, 0.06153275,
       0.05853238, 0.09809195, 0.02443584, 0.02258943, 0.09779203],
      dtype=float32)

In [29]:
# 各単語の時系列データとして扱う手法もある(ここでは紹介のみ)
# 文中の単語の順番に関する情報を考慮できる

In [30]:
from gensim.models import word2vec

# 日本語版Wikipediaで学習したWord2vec
# https://hironsan.hatenablog.com/entry/japanese-text8-corpus
sentences = word2vec.Text8Corpus("../input/ja.text8")
model = word2vec.Word2Vec(sentences)
model.wv.most_similar(["経済"])

[('財政', 0.7547577023506165),
 ('政策', 0.723992109298706),
 ('産業', 0.7084316611289978),
 ('社会', 0.6861709952354431),
 ('対外', 0.6856389045715332),
 ('資本', 0.6762275695800781),
 ('政治', 0.6757708191871643),
 ('格差', 0.6638521552085876),
 ('農業', 0.6553444862365723),
 ('軍事', 0.6500835418701172)]